# Importing Required Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers
!pip install nltk evaluate
!pip install datasets rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.5 MB/s eta 0:00:00
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environ

In [ ]:
from datasets import load_dataset
import nltk
nltk.download('punkt')
from datasets import load_dataset
from evaluate import load

# Loading Dataset, metric of evaluation

In [ ]:
raw_datasets = load_dataset("vicgalle/alpaca-gpt4")
metric = load("rouge")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset = load_dataset("vicgalle/alpaca-gpt4")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/vicgalle--alpaca-gpt4-1e85e31ce0639161/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

# PreProcessing

In [ ]:
def combine_instruction_input(example):
    example["instruction"] = example["instruction"] + " " + example["input"]
    return {"output": example["output"], "text": example["text"]}

# Apply the function to the dataset and add the new "combined_input" column
dataset = dataset.map(combine_instruction_input)

# Display the updated dataset
dataset

  0%|          | 0/52002 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 52002
    })
})

In [ ]:
from transformers import T5TokenizerFast, T5ForQuestionAnswering, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq


tokenizer = T5TokenizerFast.from_pretrained("google/t5-efficient-tiny")
model = T5ForQuestionAnswering.from_pretrained("google/t5-efficient-tiny")

Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at google/t5-efficient-tiny and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Shuffle the train set
dataset["train"] = dataset["train"].shuffle()

# Split the shuffled train set into train, test, and validation sets
datasets_train_test = dataset["train"].train_test_split(test_size=5000)
datasets_train_validation = datasets_train_test["train"].train_test_split(test_size=5000)

# Assign the split sets to the data dictionary
dataset["train"] = datasets_train_validation["train"]
dataset["validation"] = datasets_train_validation["test"]
dataset["test"] = datasets_train_test["test"]

# Print the sizes of the train, test, and validation sets
print(f"The size of the train set is {len(dataset['train'])}")
print(f"The size of the test set is {len(dataset['test'])}")
print(f"The size of the validation set is {len(dataset['validation'])}")

dataset

The size of the train set is 42002
The size of the test set is 5000
The size of the validation set is 5000


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 42002
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 5000
    })
})

In [ ]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [ ]:
prefix = "answer_question: "
max_input_length = 512
max_target_length = 512

def clean_text(text):
  sentences = nltk.sent_tokenize(text.strip())
  sentences_cleaned = [s for sent in sentences for s in sent.split("\n")]
  sentences_cleaned_no_titles = [sent for sent in sentences_cleaned
                                 if len(sent) > 0 and
                                 sent[-1] in string.punctuation]
  text_cleaned = "\n".join(sentences_cleaned_no_titles)
  return text_cleaned

def preprocess_data(examples):
  texts_cleaned = [clean_text(text) for text in examples["instruction"]]
  inputs = [prefix + text for text in examples["instruction"]]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["output"], max_length=max_target_length,padding=True ,
                       truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
import string
tokenized_datasets = dataset.map(preprocess_data, batched=True)
tokenized_datasets

  0%|          | 0/43 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 42002
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
})

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 42002
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
})

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        # Define the directory path where you want to save the model
dirr = "/kaggle/working/my_model_directory/"

# Create the directory if it doesn't exist
import os
os.makedirs(dirr, exist_ok=True)

In [ ]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

# Fine Tuning

In [ ]:
batch_size = 8
model_dir = f"/kaggle/working"

args = Seq2SeqTrainingArguments(
    model_dir,
    save_total_limit = 5,
    evaluation_strategy="steps",
    eval_steps=200,
    logging_strategy="steps",
    logging_steps=200,
    save_strategy="steps",
    save_steps=200,
    learning_rate=0.002,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
#     metric_for_best_model="rogue1",
    report_to="tensorboard"
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [ ]:
def compute_metrics(eval_pred: EvalPrediction):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids

    # Decode the predictions and labels
    decoded_preds = trainer.tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = trainer.tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE scores
    rouge_scores = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    print(rouge_scores)

#     # Extract ROUGE f1 scores
#     rouge_scores = {key: value["f"] * 100 for key, value in rouge_scores.items()}

#     # Compute mean generated length
#     prediction_lens = [np.count_nonzero(pred != trainer.tokenizer.pad_token_id) for pred in predictions]
#     mean_gen_len = np.mean(prediction_lens)

#     # Add mean generated length to metrics
#     rouge_scores["gen_len"] = mean_gen_len

    return rouge_scores

In [ ]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("google/t5-efficient-tiny")

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
200,1.637500,1.195163,0.188658,0.082730,0.166786,0.166910
400,1.352700,1.154613,0.189255,0.085125,0.167547,0.167405
600,1.328100,1.135040,0.205986,0.099838,0.182294,0.182289
800,1.292700,1.118666,0.209481,0.104214,0.185747,0.185619
1000,1.268900,1.108846,0.213118,0.108022,0.189805,0.189585
1200,1.247000,1.101533,0.219177,0.112388,0.195390,0.195170
1400,1.260400,1.090449,0.219545,0.111865,0.194772,0.194625
1600,1.243200,1.085774,0.222310,0.115640,0.197728,0.197428
1800,1.208600,1.078656,0.222588,0.114535,0.197969,0.197783
2000,1.218500,1.073141,0.224987,0.116378,0.200044,0.199958


{'rouge1': 0.1886581710879507, 'rouge2': 0.08272986218422441, 'rougeL': 0.16678559660476994, 'rougeLsum': 0.16691045551547118}


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'rouge1': 0.18925491054581278, 'rouge2': 0.08512455010249378, 'rougeL': 0.16754661752031103, 'rougeLsum': 0.16740476702389606}


...

{'rouge1': 0.2273029149718893, 'rouge2': 0.11813148242841814, 'rougeL': 0.2021610633197315, 'rougeLsum': 0.20207434865826318}


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'rouge1': 0.22795511370635466, 'rouge2': 0.1179258663478244, 'rougeL': 0.20270301820940667, 'rougeLsum': 0.20262043572615437}


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=2626, training_loss=1.2809525996060744, metrics={'train_runtime': 2513.1041, 'train_samples_per_second': 16.713, 'train_steps_per_second': 1.045, 'total_flos': 714026131113984.0, 'train_loss': 1.2809525996060744, 'epoch': 1.0})

# Loading the last saved model for predictions

In [ ]:
# model_name = "t5-efficient-tiny/checkpoint-200"
model_dir = '/kaggle/working/checkpoint-2600'

tokenizer = T5TokenizerFast.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)

max_input_length = 512

In [ ]:
text = """Write a story on moon"""

inputs = ["answer_question " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

# Conversational AI: The Future of Customer Service
predicted_title

'Once upon a time, in a circle of dawn when the moon suddenly arrived, the moon was in and the moon was a mystical and powerful moon of the moon.'